# `sc_inj` Live

In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
from lcls_live.datamaps import get_datamaps
from lcls_live.archiver import lcls_archiver_restore

from lcls_live.tools import isotime

import matplotlib.pyplot as plt
import numpy as np

import os

In [3]:
BEAMPATH = 'sc_diag0'

# Bmad model

In [4]:
from pytao import Tao
import pandas as pd

In [5]:
BEGELE = 'BEGINNING'
ENDELE = 'OTR0H04'

tao = Tao(f'-init $LCLS_LATTICE/bmad/models/sc_diag0/tao.init  -slice  {BEGELE}:{ENDELE}')
tao.cmd('place floor bpm_orbit')
tao.cmd('sc floor -10 10')

def ele_info(ele):
    dat = tao.ele_head(ele)
    dat.update(tao.ele_gen_attribs(ele))
    return dat

def ele_table(match="*"):
    ix_ele = tao.lat_list(match, "ele.ix_ele", flags="-no_slaves")
    dat = list(map(ele_info, ix_ele))
    df = pd.DataFrame(dat, index=ix_ele)
    df.L.fillna(0, inplace=True)
    df['s_center'] = df['s'] - df['L']/2
    df['s_beginning'] = df['s'] - df['L']
    return  df

df = ele_table()


# Elements with device names
devices = df[df['alias'] != '']
#devices['name alias s'.split()]

## Datamaps, and all PVs needed

In [6]:
from copy import deepcopy
def filter_datamap(dm, bmad_names):
    bnames = dm.data['bmad_name'] 
    bmad_names = set(bmad_names)
    ix = bnames[[name in bmad_names for name in bnames]].index
    dm2 = deepcopy(dm)
    dm2.data = dm.data.loc[ix]
    return dm2

In [7]:
DM0 = get_datamaps(BEAMPATH)
DM0.keys()

dict_keys(['bpms', 'cavities', 'correctors', 'quad'])

In [8]:
good_names = set(df['name'])

DM = {}
for name, dm in DM0.items():
    DM[name] = filter_datamap(dm, good_names)

DM['bpms']

TabularDataMap(data=                pvname        tao_datum    tao_factor bmad_name
0      BPMS:GUNB:314:X       orbit.x[1]  1.000000e-03     BPM1B
1      BPMS:GUNB:925:X       orbit.x[2]  1.000000e-03     BPM2B
2       BPMS:HTR:120:X       orbit.x[3]  1.000000e-03   BPM0H01
3       BPMS:HTR:320:X       orbit.x[4]  1.000000e-03   BPM0H04
23     BPMS:GUNB:314:Y       orbit.y[1]  1.000000e-03     BPM1B
24     BPMS:GUNB:925:Y       orbit.y[2]  1.000000e-03     BPM2B
25      BPMS:HTR:120:Y       orbit.y[3]  1.000000e-03   BPM0H01
26      BPMS:HTR:320:Y       orbit.y[4]  1.000000e-03   BPM0H04
46  BPMS:GUNB:314:TMIT  orbit.charge[1]  1.602177e-19     BPM1B
47  BPMS:GUNB:925:TMIT  orbit.charge[2]  1.602177e-19     BPM2B
48   BPMS:HTR:120:TMIT  orbit.charge[3]  1.602177e-19   BPM0H01
49   BPMS:HTR:320:TMIT  orbit.charge[4]  1.602177e-19   BPM0H04, pvname='pvname', element='tao_datum', attribute='', factor='tao_factor', offset='', bmad_format='! No equivalent Bmad format for: set data {element

In [9]:
list(DM['correctors'].data['pvname'])

['XCOR:GUNB:293:BACT',
 'XCOR:GUNB:388:BACT',
 'XCOR:GUNB:513:BACT',
 'XCOR:GUNB:713:BACT',
 'XCOR:GUNB:927:BACT',
 'XCOR:L0B:0185:BACT',
 'XCOR:HTR:148:BACT',
 'XCOR:HTR:288:BACT',
 'YCOR:GUNB:293:BACT',
 'YCOR:GUNB:388:BACT',
 'YCOR:GUNB:513:BACT',
 'YCOR:GUNB:713:BACT',
 'YCOR:GUNB:927:BACT',
 'YCOR:L0B:0185:BACT',
 'YCOR:HTR:129:BACT',
 'YCOR:HTR:309:BACT']

In [10]:
# datamaps to exclude
DENYLIST = [
    #'bpms',
    'cavities',
    'correctors',
    #'quad',
]

In [11]:
# PVs needed
PVLIST =  []
for name, dm in DM.items():
    if name in DENYLIST:
        continue
    PVLIST.extend(dm.pvlist)
PVLIST = list(set(PVLIST))
len(PVLIST)

17

## EPICS

In [12]:
import epics
from epics import caget_many, caget
from time import sleep, time

def caget_dict(pvlist):
    return dict(zip(pvlist, caget_many(pvlist)))

In [13]:
MONITOR = {pvname:epics.PV(pvname) for pvname in PVLIST}
sleep(1) # Wait for all to connect

In [14]:
def get_pvdata():   
    itime = isotime()
    pvdata =  {k:MONITOR[k].get() for k in MONITOR}
    return pvdata
PVDATA = get_pvdata()

PVDATA

{'BPMS:HTR:120:X': 0.0,
 'QUAD:HTR:300:BACT': -1.149942356212156,
 'BPMS:GUNB:314:X': 0.0,
 'QUAD:L0B:0185:BACT': 0.00287317367486943,
 'BPMS:GUNB:314:Y': 0.0,
 'QUAD:HTR:120:BACT': -1.9341131855324296,
 'BPMS:HTR:320:X': 0.0,
 'BPMS:GUNB:925:TMIT': 0.0,
 'BPMS:HTR:320:Y': 0.0,
 'BPMS:HTR:320:TMIT': 0.0,
 'QUAD:HTR:140:BACT': 2.0053450063492724,
 'BPMS:GUNB:314:TMIT': 0.0,
 'BPMS:HTR:120:TMIT': 0.0,
 'BPMS:GUNB:925:X': 0.0,
 'QUAD:HTR:320:BACT': 0.8273343077135241,
 'BPMS:HTR:120:Y': 0.0,
 'BPMS:GUNB:925:Y': 0.0}

In [15]:
# Check for bad PVs
bad = set()
for k, v in PVDATA.items():
    if v is None:
        bad.add(k.split(':')[2])
        print(k)
assert len(bad) == 0

# Tao conveniences

In [16]:
def tao_commands(pvdata):
    cmds = []
    for name, dm in DM.items():
        cmds.extend(dm.as_tao(pvdata))
    return cmds

In [17]:
def save_cmds(cmds, filename='cmds.tao'): # Write to file for running with vanilla Tao
    with open(filename, 'w') as f:
        f.write('set global lattice_calc_on = F\n')
        f.write('set global plot_on = F\n')    
        for cmd in CMDS:
            f.write(cmd+'\n')
        f.write('set global lattice_calc_on = T\n')        
        f.write('set global plot_on = T\n')   

## Form commands using PVDATA and datamaps

In [18]:
tao_commands(PVDATA)[0:5]

['set data orbit.x[1]|meas  = 0.001 * 0.0',
 'set data orbit.x[2]|meas  = 0.001 * 0.0',
 'set data orbit.x[3]|meas  = 0.001 * 0.0',
 'set data orbit.x[4]|meas  = 0.001 * 0.0',
 'set data orbit.y[1]|meas  = 0.001 * 0.0']

# Energy setup

In [19]:
# Measured energy
energy_meas = caget('BEND:HTR:480:BACT') * 1e9
energy_meas/ 1e6 # MeV

59.91548872666886

In [20]:
tao.cmd('set ele quad::* field_master = T')

[]

In [21]:
cmds = f"""
set lattice model=design
vv
vd

set dat L0B.energy|meas = {energy_meas}

use var cavities[8]
use dat L0B.energy
run
""".strip().split('\n')
for cmd in cmds:
    tao.cmd(cmd)

In [22]:
%%tao
sho lat -at alias ENDL0B:OTR0H04

-------------------------
Tao> sho lat -at alias ENDL0B:OTR0H04
# Values shown are for the Exit End of each Element:
# Index  name    key                      s       l  alias
    119  ENDL0B  Marker              18.175   0.000
    120  BEGHTR  Marker              18.175   0.000
    121  RFB0H00 Pipe                18.175   0.000
    122  D0H00A  Drift               18.231   0.056
    123  PC0H00  Ecollimator         18.291   0.060  COLL:HTR:103
    124  D0H00B  Drift               18.583   0.292
    125  Q0H01#1 Quadrupole          18.645   0.062
    126  BPM0H01 Monitor             18.645   0.000  BPMS:HTR:120
    127  Q0H01#2 Quadrupole          18.707   0.062
    128  D0H01A  Drift               18.921   0.214
    129  YC0H01  Vkicker             18.921   0.000  YCOR:HTR:129
    130  D0H01B  Drift               19.135   0.214
    131  Q0H02   Quadrupole          19.260   0.124  QUAD:HTR:140
    132  D0H02A  Drift               19.464   0.205
    133  XC0H01  Hkicker             19.

# Continuous loop

In [23]:
%%tao
use dat orbit.x
use dat orbit.y

-------------------------
Tao> use dat orbit.x
  orbit.x[1:23]                                  Using: 1
  orbit.y[1:23]                                  Using:
  orbit.charge[1:23]                             Using:
-------------------------
Tao> use dat orbit.y
  orbit.x[1:23]                                  Using: 1
  orbit.y[1:23]                                  Using: 1
  orbit.charge[1:23]                             Using:
-------------------------
Tao> 


In [24]:
def run1():
    #sleep(.001)
    t1 = time()
    pvdata = get_pvdata()
    cmds = tao_commands(pvdata)
    tao.cmd('set global plot_on = F;set global lattice_calc_on = F')
    tao.cmds(cmds); # Apply

    tao.cmd('set global lattice_calc_on = T')
    tao.cmd('set global plot_on = T')
    #toggle_beam()
    
    dt = time()-t1
    #print(dt)    


In [25]:
%%time
run1()

CPU times: user 17.1 ms, sys: 1.94 ms, total: 19.1 ms
Wall time: 27.1 ms


In [26]:
# mat2 x, y for PyEmittance
def get_mats():
    mat6 = tao.matrix('Q0H01#2', 'OTR0H04')['mat6']
    mat2x = mat6[0:2, 0:2]
    mat2y = mat6[2:4, 2:4]
    return mat2x, mat2y


get_mats()

(array([[-5.04075351,  4.00431172],
        [-0.90991524,  0.52444228]]),
 array([[ 4.23518477,  5.6118353 ],
        [-0.24974114, -0.09480251]]))

CA.Client.Exception...............................................
    Context: "localhost:24669"
    Source File: ../tcpiiu.cpp line 926
    Current Time: Sat Sep 24 2022 09:43:00.359625000
..................................................................


In [ ]:
# Run forever
#while True:
#    run1()